<a href="https://colab.research.google.com/github/Shashi-Sekar/CoviProtec/blob/master/CNN_SLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2

import os
import pickle
from os.path import join, exists
import argparse
from tqdm import tqdm
from google.colab.patches import cv2_imshow
import numpy as np

from datetime import timedelta
import numpy as np
import tensorflow as tf
import time

In [ ]:
LSAvideos_loc = "/content/drive/MyDrive/LSAvideos/"
files = os.listdir(LSAvideos_loc)
files

['all',
 'Sample_Videos',
 'Frames_sample',
 'Test_Videos',
 'Train_Frames',
 'Test_Frames',
 'CNN_Train_Output_Part2',
 'CNN_Train_Output_Part1']

In [ ]:
#-------------FOR Training FRAMES-----------------
train_cnn_output_loc = "/content/drive/MyDrive/LSAvideos/Train_Frames/"
extracted_train_frames = os.listdir(train_cnn_output_loc)
print ("Number of frames extracted is: ", len(extracted_train_frames))

train_img_name_vector = []

for extracted_train_frame in extracted_train_frames:
  train_img_name_vector.append(train_cnn_output_loc + extracted_train_frame)

OSError: ignored

In [ ]:
train_img_name_vector.sort()
print(len(train_img_name_vector))
print(train_img_name_vector[0])
print(train_img_name_vector[len(train_img_name_vector) - 1])

In [ ]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

A = [1,2,3,4,5,6]
B, C = split_list(A)

In [ ]:
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

B, C, D = split_list(A, 3)

In [ ]:
sample_output_loc = "/content/drive/MyDrive/LSAvideos/CNN_Train_Output_Part1"
os.chdir(sample_output_loc)


FileNotFoundError: ignored

In [ ]:
image_model = tf.keras.applications.InceptionV3(include_top = False, weights = 'imagenet', pooling='avg')

new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)
image_features_extract_model.summary()

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [ ]:
cnn_train = sorted(set(train_img_name_vector))

image_dataset = tf.data.Dataset.from_tensor_slices(cnn_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

In [ ]:
sample_output_loc = "/content/drive/MyDrive/LSAvideos/Train_Frames/"
os.chdir(sample_output_loc)

for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)
  
  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

**EXTRAS**

In [ ]:
#-------------FOR SAMPLE FRAMES-----------------
sample_output_loc = "/content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames"
extracted_sample_frames = os.listdir(sample_output_loc)
print ("Number of frames extracted from 25 videos is: ", len(extracted_sample_frames))

sample_img_name_vector = []

for extracted_sample_frame in extracted_sample_frames:
  sample_img_name_vector.append(sample_output_loc + "/" + extracted_sample_frame)#[join(sample_output_loc, extracted_sample_frame)])
  break

Number of frames extracted from 25 videos is:  472


In [ ]:
for i, val in enumerate(sample_img_name_vector):
  print (i, val)

0 /content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames/001_001_001_1.jpg


In [ ]:
print (sample_img_name_vector[0])
#print (sample_img_name_vector[1])

/content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames/001_001_001_1.jpg


In [ ]:
image_model = tf.keras.applications.InceptionV3(include_top = False, weights = 'imagenet')

new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)
image_features_extract_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_658 (Conv2D)            (None, None, None,   864         ['input_8[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_658 (Batch  (None, None, None,   96         ['conv2d_658[0][0]']             
 Normalization)                 32)                                                         

In [ ]:
def list_to_set_conversion(list1):
    se = set()
    for x in list1:
        se.add(x)
    return se

img_name_vector_set = list_to_set_conversion(sample_img_name_vector)

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [ ]:
cnn_train = sorted(set(sample_img_name_vector))

image_dataset = tf.data.Dataset.from_tensor_slices(cnn_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

In [ ]:
image_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None))>

In [ ]:
sample_output_loc = "/content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames/Output_cnn/"
os.chdir(sample_output_loc)

for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

100%|██████████| 1/1 [00:00<00:00,  1.55it/s]


In [ ]:
sample_feature_frame = np.load("/content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames/Output_cnn/001_001_001_1.jpg.npy")
sample_feature_frame.shape

(64, 2048)

In [ ]:
sample_feature_frame

array([[0.        , 0.64463246, 0.4670758 , ..., 0.666816  , 0.        ,
        0.        ],
       [0.35260943, 0.32292733, 0.37959597, ..., 0.96097684, 0.        ,
        0.        ],
       [0.03313947, 0.61282325, 0.47035432, ..., 1.655065  , 0.        ,
        0.38755324],
       ...,
       [0.        , 0.        , 0.        , ..., 0.05598574, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04144365, 0.        ,
        0.        ],
       [0.        , 0.        , 0.1393767 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [ ]:
image_model = tf.keras.applications.InceptionV3(weights = 'imagenet')
#include_top = False, 
new_input = image_model.input
hidden_layer = image_model.layers[-2].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)
image_features_extract_model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1222 (Conv2D)           (None, 149, 149, 32  864         ['input_14[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_1222 (Batc  (None, 149, 149, 32  96         ['conv2d_1222[0][0]']            
 hNormalization)                )                                                          

In [ ]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [ ]:
from keras.preprocessing import image
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)
    return x

In [ ]:
x = preprocess(sample_img_name_vector[0])
tryi = image_features_extract_model.predict(preprocess(sample_img_name_vector[0]))
tryi.shape

(1, 2048)

In [ ]:
tryi

array([[0.34167552, 0.9336555 , 0.16649722, ..., 0.74608576, 0.3919207 ,
        0.39264235]], dtype=float32)

In [ ]:
sample_feature_frame

array([[0.        , 0.64463246, 0.4670758 , ..., 0.666816  , 0.        ,
        0.        ],
       [0.35260943, 0.32292733, 0.37959597, ..., 0.96097684, 0.        ,
        0.        ],
       [0.03313947, 0.61282325, 0.47035432, ..., 1.655065  , 0.        ,
        0.38755324],
       ...,
       [0.        , 0.        , 0.        , ..., 0.05598574, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04144365, 0.        ,
        0.        ],
       [0.        , 0.        , 0.1393767 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [ ]:
cnn_train = sorted(set(sample_img_name_vector))

image_dataset = tf.data.Dataset.from_tensor_slices(cnn_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

In [ ]:
image_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None))>

In [ ]:
sample_output_loc = "/content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames/Output_cnn/"
os.chdir(sample_output_loc)

for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (1, batch_features.shape[1]))
  print(batch_features.shape)
  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]

(1, 2048)


In [ ]:
sample_feature_frame = np.load("/content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames/Output_cnn/001_001_001_1.jpg.npy")
sample_feature_frame.shape
sample_feature_frame

array([0.4379494 , 0.9418236 , 0.16054806, ..., 0.82999206, 0.3377397 ,
       0.43619168], dtype=float32)

In [ ]:
image_model = tf.keras.applications.InceptionV3(include_top = False, weights = 'imagenet', pooling='avg')

new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)
image_features_extract_model.summary()

87924736/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                  

In [ ]:
cnn_train = sorted(set(sample_img_name_vector))

image_dataset = tf.data.Dataset.from_tensor_slices(cnn_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

In [ ]:
sample_output_loc = "/content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames/Output_cnn/"
os.chdir(sample_output_loc)

for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)
  #batch_features = tf.reshape(batch_features,
                              #(1, batch_features.shape[1]))
  print(batch_features.shape)
  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

100%|██████████| 1/1 [00:00<00:00,  1.67it/s]

(1, 2048)


In [ ]:
sample_feature_frame = np.load("/content/drive/MyDrive/LSAvideos/Frames_sample/testing_sample_frames/Output_cnn/001_001_001_1.jpg.npy")
print(sample_feature_frame.shape)
print(sample_feature_frame)

(2048,)
[0.4379494  0.9418236  0.16054806 ... 0.82999206 0.3377397  0.43619168]


In [ ]:
target_label = [['001', 'Opaque'], ['003', 'Green'], ['004', 'Yellow'], ['005', 'Bright'], ['006', 'Light-blue'], ['007', 'Colors'], ['008', 'Red'], ['009', 'Women'],
                ['010', 'Enemy'], ['011', 'Son'], ['012', 'Man'], ['013', 'Away'], ['014', 'Drawer'], ['015', 'Born'], ['016', 'Learn'], ['017', 'Call'], ['018', 'Skimmer'], 
                ['019', 'Bitter'], ['020', 'Sweet milk'], ['021', 'Milk'], ['022', 'Water'], ['023', 'Food'], ['024', 'Argentina'], ['025', 'Uruguay'], ['026', 'Country']]

In [ ]:
for i, val in enumerate(target_label):
  print(val[0], val[1])

001 Opaque
003 Green
004 Yellow
005 Bright
006 Light-blue
007 Colors
008 Red
009 Women
010 Enemy
011 Son
012 Man
013 Away
014 Drawer
015 Born
016 Learn
017 Call
018 Skimmer
019 Bitter
020 Sweet milk
021 Milk
022 Water
023 Food
024 Argentina
025 Uruguay
026 Country


In [ ]:
def load_labels(label_list):
    label = {}
    
    count = 0

    for i, val in enumerate(label_list):
        label[val[1]] = count
        count += 1

    return label

In [ ]:
labels = load_labels(target_label)
print(len(labels))
labels

25


{'Argentina': 22,
 'Away': 11,
 'Bitter': 17,
 'Born': 13,
 'Bright': 3,
 'Call': 15,
 'Colors': 5,
 'Country': 24,
 'Drawer': 12,
 'Enemy': 8,
 'Food': 21,
 'Green': 1,
 'Learn': 14,
 'Light-blue': 4,
 'Man': 10,
 'Milk': 19,
 'Opaque': 0,
 'Red': 6,
 'Skimmer': 16,
 'Son': 9,
 'Sweet milk': 18,
 'Uruguay': 23,
 'Water': 20,
 'Women': 7,
 'Yellow': 2}

In [ ]:
#One Hot encoding of Label
actual_label = labels['Learn']
y = []

y.append(actual_label)
y = np.array(y)
x = tf.keras.utils.to_categorical(y, 25, dtype ="uint8")
x

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]], dtype=uint8)

In [ ]:
num_classes = 25
label_encoding = []

for i, val in enumerate(target_label):
  actual_label = labels[val[1]]
  
  y = []
  y.append(actual_label)
  y = np.array(y)
  
  label_encoding.append(tf.keras.utils.to_categorical(y, 25, dtype ="uint8"))

In [ ]:
label_encoding

[array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, 0

In [ ]:
rows, cols = (25, 2)
target_label_encoding = [[0]*cols]*rows
target_label_encoding

[[0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0]]

In [ ]:
target_label_encoding = []
for i in range(rows):
  target_label_encoding.append([target_label[i][0], label_encoding[i]])

In [ ]:
target_label_encoding

[['001',
  array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]], dtype=uint8)],
 ['003',
  array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]], dtype=uint8)],
 ['004',
  array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]], dtype=uint8)],
 ['005',
  array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]], dtype=uint8)],
 ['006',
  array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]], dtype=uint8)],
 ['007',
  array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]], dtype=uint8)],
 ['008',
  array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]], dtype=uint8)],
 ['009',
  array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]], dtype=uint8)],
 ['010',
  array([[0, 0, 0, 0, 0